In [1]:
# Importing basic libraries
import pandas as pd
import numpy as np
import spacy

In [2]:
df = pd.read_csv('dataset/review.csv')  #reading reviews dataset

In [3]:
print(df.shape)
df.head()

(14675, 2)


,sentiment,review
0,1,Good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...
3,1,Good
4,0,The worst phone everThey have changed the last...


In [4]:
df.isnull().sum() #Checking Null Values

sentiment    0
review       0
dtype: int64

In [5]:
nlp = spacy.load('en_core_web_sm') #Loading spacy english

In [6]:
# Text Preprocessing
import string
punct = string.punctuation
def text_clean(text):
    text = text.lower()  #Convert text in lower case
    punc_removed = [char for char in text if char not in punct]  #Removing Punctuations
    punc_removed_join = ''.join(punc_removed) 
    
    removal=['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']  #get Noun phrase
    text_out = []
    doc= nlp(punc_removed_join)
    for token in doc:
        if token.is_stop == False and token.is_alpha and len(token)>2 and token.pos_ not in removal:
            lemma = token.lemma_            #lemmatization of token word
            text_out.append(lemma)
            
    return text_out

In [7]:
df['clean_doc'] = df['review'].apply(text_clean) #Apply Preprocessing of text
df.head()

,sentiment,review,clean_doc
0,1,Good but need updates and improvements,"[good, need, update, improvement]"
1,0,"Worst mobile i have bought ever, Battery is dr...","[bad, mobile, buy, battery, drain, like, hell,..."
2,1,when I will get my 10% cash back.... its alrea...,"[cash, january]"
3,1,Good,[good]
4,0,The worst phone everThey have changed the last...,"[bad, phone, everthey, change, phone, problem,..."


In [8]:

# print(df['review'][1])
# print(len(df['review'][1]))
# print(df['clean_doc'][1])
# print(len(df['clean_doc'][1]))
# print("percent token : %0.2f" %(float(len(df['clean_doc'][1]) * 100)/len(df['review'][1])))

In [9]:
# Getting text in list
val = []
for i in range(len(df)):
    val.append(df['clean_doc'][i])
# val

In [10]:
# import gensim library
from gensim import models,corpora
dictionary = corpora.Dictionary(val)
dictionary

In [11]:
bow_corpus = [dictionary.doc2bow(doc) for doc in val]   #Creating bag of words
# bow_corpus

In [12]:
# process gensim model for topic identification
import gensim
lda_model =  gensim.models.LdaMulticore(bow_corpus, num_topics = 5, id2word = dictionary, passes = 10,workers = 2)

In [13]:
lda_model

In [14]:
# Printing Topics and Its keywords
num_topics = 5
for i in range(num_topics):
    tt = lda_model.get_topic_terms(i,10)
    topic = ', '.join([dictionary[pair[0]] for pair in tt])
    print("TOPIC: {} \nTOPIC WORDS : {}".format(i+1, topic ))
    print()

TOPIC: 1 
TOPIC WORDS : phone, amazon, buy, get, problem, network, lenovo, issue, sim, mobile

TOPIC: 2 
TOPIC WORDS : good, phone, nice, product, mobile, price, awesome, excellent, performance, feature

TOPIC: 3 
TOPIC WORDS : camera, good, phone, quality, battery, work, dual, feature, bad, price

TOPIC: 4 
TOPIC WORDS : battery, phone, charge, lenovo, note, heating, problem, issue, heat, camera

TOPIC: 5 
TOPIC WORDS : bad, work, product, mobile, buy, waste, lenovo, money, charger, month



In [15]:
# for idx, topic in lda_model.print_topics():
#     print("Topic: {} \n Words: {}".format(idx, topic ))
#     print("\n")

In [16]:
# Visualize the topics
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.067227 -0.117484       1        1  28.898624
2      0.059027 -0.156040       2        1  25.603518
0     -0.126397  0.026747       3        1  23.566181
1      0.256890  0.091967       4        1  13.103631
4     -0.122292  0.154811       5        1   8.828050, topic_info=        Term         Freq        Total Category  logprob  loglift
0       good  5551.000000  5551.000000  Default  30.0000  30.0000
195     nice  1164.000000  1164.000000  Default  29.0000  29.0000
111   camera  2958.000000  2958.000000  Default  28.0000  28.0000
99   product  1881.000000  1881.000000  Default  27.0000  27.0000
6        bad  1704.000000  1704.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
15    expect    48.749676   248.946426   Topic5  -5.5504   0.7967
166      day    56.865711   819.528320   Topic5  -5.3964  -0.2408
319    sound    51.758457   443.295380   Topic5  -5.4905   0.2796
77      poor    53.736481   712.560791   Topic5  -5.4530  -0.1575
54      note    48.233078  1159.605713   Topic5  -5.5610  -0.7526

[393 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
594       3  0.974254   accessory
2483      2  0.914792    accurate
472       2  0.951463      adjust
799       4  0.965875  affordable
824       3  0.949474      airtel
...     ...       ...         ...
175       3  0.199739       worth
175       4  0.527613       worth
175       5  0.033918       worth
662       2  0.960305        zoom
662       3  0.033114        zoom

[764 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 2, 5])